In [1]:
import os
from datetime import datetime
# Make logging folder named according to current time
start_time_str = datetime.today().strftime('%d_%m_%Y__%H_%M_%S')
log_dir = './logs/'f"log_{start_time_str}"
os.makedirs(log_dir)

# Initialize logger that adds to text file in current logging folder
from logging_utils import *


# Initialize logger that adds to text file in current logging folder
from logging_utils import *
init_logger(save_dir=log_dir)
logger = get_logger()

In [2]:
import pathlib
import subprocess
# Get the absolute path to your repository, 
# no matter where you are running this code from

repo_path = os.getcwd() 


git_branch = subprocess.check_output(
    ["git", "-C", repo_path, "rev-parse", "--abbrev-ref", "HEAD"]).strip().decode('UTF-8')

git_commit_short_hash = subprocess.check_output(
    ["git", "-C", repo_path, "describe", "--always"]).strip().decode('UTF-8')


In [3]:
from src_dir import *

import numpy as np

dim=125
x0=np.squeeze(np.zeros((dim,1)))
x0Type='Zero Vector'

# FD_StencilMat=np.zeros((dim,dim))
# b = np.ones(dim-1)
# c = -2*np.ones(dim)
# np.fill_diagonal(FD_StencilMat[1:], b)
# np.fill_diagonal(FD_StencilMat[:,1:], b)
# np.fill_diagonal(FD_StencilMat, c)
# A=FD_StencilMat
# A=np.asmatrix(A)
A = mk_laplace_1d(dim)
AType='1D Laplacian'

retrain_freq=1
e1 = 1e-4
e2 = 1e-7
nmax_iter = 50
restart   = 64

sigma=0.02
DomainL=-5.0
DomainR=5.0
x=np.linspace(DomainL,DomainR,dim)



n_steps =500
InputDim=dim
HiddenDim=dim
OutputDim=dim

In [4]:
# Initial log message
logger.info("Benchmark 12 dim problem using op func. ")
#  The source is a narrow gaussian that depends on the integer 'time' in a nonlinear way. The gaussian is on the interval [-5,5], and constrained to move on [-3,3]. ")



# Set up parameter class
class MyParamsClass():
    def __init__(self, AType,dim,nmax_iter,restart,retrain_freq,e1,e2,x0Type,n_steps,InputDim,HiddenDim,OutputDim,sigma,DomainL,DomainR,git_branch,git_commit_short_hash):
        self.AType = AType
        self.dim = dim
        self.nmax_iter = nmax_iter
        self.restart = restart
        self.n_steps =n_steps
        self.retrain_freq=retrain_freq
        self.e1=e1
        self.e2=e2
        self.x0Type=x0Type
        self.InputDim=InputDim
        self.HiddenDim=HiddenDim
        self.OutputDim=OutputDim
        self.sigma=sigma
        self.DomainL=DomainL
        self.DomainR=DomainR
        self.git_branch=git_branch
        self.git_commit_short_hash=git_commit_short_hash





import os
import json

params = MyParamsClass(AType,dim,nmax_iter,restart,retrain_freq,e1,e2,x0Type,n_steps,InputDim,HiddenDim,OutputDim,sigma,DomainL,DomainR,git_branch,git_commit_short_hash)
sorted_params_dict = {k: params.__dict__[k] for k 
                      in sorted(params.__dict__.keys())}

params_filepath = log_dir+'/params.json'
json.dump(sorted_params_dict, open(params_filepath, 'w'), indent=4)

2020-06-09 19:29:20 [INFO] <ipython-input-4-6ec73637bfb3> 2 - Benchmark 12 dim problem using op func. 


In [5]:
# @nn_preconditioner(retrain_freq=retrain_freq, debug=True,InputDim=InputDim,HiddenDim=HiddenDim,HiddenDim2=HiddenDim2,OutputDim=OutputDim)
# def MLGMRES(A, b, x0, e, nmax_iter,IterErr0_sum,IterErr0,ProbCount,Add,restart,debug):
#     return GMRES(A, b, x0, e, nmax_iter,restart, debug)



@timer
@cnn_preconditionerOnline_timed(retrain_freq=retrain_freq, debug=True,InputDim=InputDim,HiddenDim=HiddenDim,OutputDim=OutputDim)
def MLGMRES(A, b, x0, e, nmax_iter,ML_GMRES_Time_list,ProbCount,restart,debug,blist,reslist,Err_list):
    return GMRES(A, b, x0, e, nmax_iter,restart, debug)



@timer
def GMRES_timed(A, b, x0, e, nmax_iter,restart,debug):
    return GMRES(A, b, x0, e, nmax_iter,restart, debug)

In [ ]:
from src_dir import *



ML_GMRES_Time_list=[] 
Err_list=[]


trainTime_list=[]
NonML_Err_List=[]

run_time_ML_list=[]
run_time_list=[]

run_time_ML_listP1=[]
run_time_ML_listP2=[]

GmresRunTimeOriginal=[]
GmresRunTimeOriginal_2=[]


debug=True

blist=[]
reslist=[]



for ProbIdx in range(n_steps):

    loc=4*np.sin(0.5*ProbIdx*np.abs(np.cos(0.5*ProbIdx)))
#     b=Gauss_pdf(x,loc,sigma)
    b=(np.sin(ProbIdx)+.0001)*np.maximum(x-loc,0)
    b_norm=np.linalg.norm(b)



    OutList1,run_time1_ML=MLGMRES(A, b/b_norm, x0, e1, nmax_iter,ML_GMRES_Time_list,ProbIdx,restart,debug,blist,reslist,Err_list)  # normalize b for optimal NN performance.
    Out=OutList1[0]
    ML_GMRES_Time_list=OutList1[1]
    trainTime_list.append(OutList1[2])
    blist=OutList1[3]
    reslist=OutList1[4]
    Err_list=OutList1[5]


    ML_Out2,run_time2_ML=GMRES_timed(A, b/b_norm, Out[-1], e2, nmax_iter,restart, debug)  # normalize b for optimal NN performance.

    
    run_time_ML_listP1.append(run_time1_ML)
    run_time_ML_listP2.append(run_time2_ML)
    run_time_ML_list.append(run_time1_ML+run_time2_ML)




    NonML_Out1,run_time1=GMRES_timed(A, b/b_norm, x0, e1, nmax_iter,restart, debug)  # normalize b for optimal NN performance.
    NonML_Err=resid(A, NonML_Out1, b/b_norm)
    NonML_Err_List.append(NonML_Err[10])
    GmresRunTimeOriginal.append(run_time1)
   
    NonML_Out2,run_time2=GMRES_timed(A, b/b_norm, NonML_Out1[-1], e2, nmax_iter,restart, debug)  # normalize b for optimal NN performance.
    run_time_list.append(run_time1+run_time2)
    GmresRunTimeOriginal_2.append(run_time2)

    print(ProbIdx)


run_time=sum(run_time_list)
run_time_ML=sum(run_time_ML_list)
trainTime_total=sum(trainTime_list)


logger.info("Runtime of Non-decorated version is:")
logger.info(run_time)

logger.info("Runtime of MLGMRES decorator is:")
logger.info(run_time_ML)


logger.info("Runtime of training (backprop) is:")
logger.info(trainTime_total)



0
1
Final loss: 7255.71142578125
parameters 31250
Initial Training
2
size 52
0.29239550338018794 1.0067487400056774
0.33859443198889494 0.6300515414914116 0.29239550338018794 0.9957254249702396
3
size 52
1.1467397285312901 1.0067487400056774
poor prediction,using initial x0
0.6638138000271283 0.4402301564987283 0.9654725687010541 0.6428550203175197
4
size 52
1.06880555863422 1.0067487400056774
poor prediction,using initial x0
0.4156340599874966 0.5147580376748616 0.8981946663913993 0.7503942031120312
5
size 52
2.027264179907995 1.0067487400056774
poor prediction,using initial x0
0.5360621720319614 0.47268076400117326 1.0048953498739603 0.7186875794908806
6
size 52
0.5533265793042965 1.0067487400056774
0.4932582460460253 0.4885261160088703 0.5533265793042965 0.7902395220866505
InnerProd [[ 1.          0.98188841 -0.9659102 ]
 [ 0.98188841  1.         -0.99674935]
 [-0.9659102  -0.99674935  1.        ]]
retraining
1
Final loss: 370.7779541015625
parameters 31250
7
size 52
0.2045820699622

0.22110946098109707 0.37110234251711516 0.10513856337298048 0.2982453748407386
57
size 52
0.13913193084293637 1.0081862607814258
0.26603675802471116 0.36862668399699033 0.13913193084293637 0.29826930372905663
58
size 52
0.09892415650592726 1.0081862607814258
0.3950760720181279 0.3523801760771312 0.09892415650592726 0.2790973672294736
InnerProd [[ 0.99999994 -0.8988217   0.9979737 ]
 [-0.8988217   0.99999994 -0.91746676]
 [ 0.9979737  -0.91746676  0.9999999 ]]
retraining
1
Final loss: 666.2495727539062
parameters 31250
59
size 52
0.07131442650811501 1.0081862607814258
0.28197720198659226 0.3435225928388536 0.07131442650811501 0.2558853609095815
60
size 52
0.20283489574596242 1.0081862607814258
0.35801665199687704 0.3438940086797811 0.20283489574596242 0.2543577159591118
61
size 52
0.24460030427548565 1.0081862607814258
0.3557433990063146 0.3473032921599224 0.24460030427548565 0.25773406972055535
62
size 52
0.1683761865937586 1.0081862607814258
0.3331945359823294 0.34290839500026776 0.16

110
size 52
0.0935707530070755 1.0081862607814258
0.13183743599802256 0.2665933731617406 0.0935707530070755 0.10870402263775432
111
size 52
0.06663089227953295 1.0081862607814258
0.2842040810501203 0.26169747876003385 0.06663089227953295 0.11175608336443782
112
size 52
0.0844579267385651 1.0081862607814258
0.2690955320140347 0.2640064157219604 0.0844579267385651 0.11133007508927725
InnerProd [[1.0000002 0.9172837 0.9459655]
 [0.9172837 1.0000001 0.99443  ]
 [0.9459655 0.99443   1.       ]]
retraining
1
Final loss: 207.06387329101562
parameters 31250
113
size 52
0.10420122314240614 1.0081862607814258
0.17341863800538704 0.26519930192269386 0.10420122314240614 0.11061916359639533
114
size 52
0.084153730512647 1.0081862607814258
0.19755670701852068 0.2644039024040103 0.084153730512647 0.1098365953351366
115
size 52
0.06588344533620476 1.0081862607814258
0.10673290299018845 0.2540918100834824 0.06588344533620476 0.10951968796552528
116
size 52
0.1849059523555927 1.0081862607814258
0.286718

0.14933792396914214 0.21359434264013544 0.08300466166967091 0.09386352182172372
164
size 52
0.05018100804064071 1.0081862607814258
0.146631169016473 0.21169981931801884 0.05018100804064071 0.09234871715814481
165
size 52
0.07239559194490064 1.0081862607814258
0.17008542700204998 0.21066613763803615 0.07239559194490064 0.0912059167339011
166
size 52
0.17814897501050733 1.0081862607814258
0.22570999298477545 0.21061271207872778 0.17814897501050733 0.09229184151412163
167
size 52
0.11658040591083424 1.0081862607814258
0.1938292080303654 0.2082105432380922 0.11658040591083424 0.09588843402622181
168
size 52
0.031251429815686585 1.0081862607814258
0.14604777499334887 0.20464636627817526 0.031251429815686585 0.0911677480511497
169
size 52
0.07927776998330666 1.0081862607814258
0.17205917899264023 0.20313452259637416 0.07927776998330666 0.08889157280033093
170
size 52
0.046764418135684375 1.0081862607814258
0.1711803670041263 0.20270288039697335 0.046764418135684375 0.08867170816539179
171
si

0.2812164889764972 0.17099095171783119 0.04402051360856232 0.06501031406632259
InnerProd [[ 1.         0.9652201 -0.9861267]
 [ 0.9652201  1.0000001 -0.9942848]
 [-0.9861267 -0.9942848  1.0000001]]
retraining
1
Final loss: 137.83663940429688
parameters 31250
217
size 52
0.045256910479456625 1.0081862607814258
0.17995493701891974 0.17683832151582465 0.045256910479456625 0.06560669719583158
218
size 52
0.04811331672316321 1.0081862607814258
0.22487163200275972 0.17941752411657944 0.04811331672316321 0.06659449343557303
219
size 52
0.05078464046148338 1.0081862607814258
0.17938713397597894 0.18480272027663885 0.05078464046148338 0.0642087817386758
220
size 52
0.07174895045976584 1.0081862607814258
0.22131729603279382 0.18664015219546853 0.07174895045976584 0.06487844913274368
InnerProd [[ 0.99999994  0.98357743 -0.99798566]
 [ 0.98357743  1.0000002  -0.97150636]
 [-0.99798566 -0.97150636  1.0000001 ]]
retraining
1
Final loss: 130.09674072265625
parameters 31250
221
size 52
0.0315621913565

268
size 52
0.012761393196836118 1.0081862607814258
0.13872840302065015 0.19027492659632117 0.012761393196836118 0.047483218925632766
269
size 52
0.09471645372054555 1.0081862607814258
0.1839050049893558 0.18902234639739618 0.09471645372054555 0.0470371570951514
270
size 52
0.015222436162718884 1.0081862607814258
0.11430855799699202 0.18731322787702084 0.015222436162718884 0.04908567811017315
271
size 52
0.13436474579761537 1.0081862607814258
0.16772208001930267 0.18051398227689788 0.13436474579761537 0.04669487010768744
272
size 52
0.06830942442398716 1.0081862607814258
0.13213158602593467 0.17815296755637974 0.06830942442398716 0.049808521729716594
273
size 52
0.04578092367712293 1.0081862607814258
0.0871509540011175 0.17752356643555686 0.04578092367712293 0.05170844615917272
274
size 52
0.0414009406598833 1.0081862607814258
0.11394037201534957 0.1751178724761121 0.0414009406598833 0.05196357119855857
275
size 52
0.07492472120226276 1.0081862607814258
0.13272221898660064 0.1750723759

321
size 52
0.13694577040087055 1.0081862607814258
0.1304671279503964 0.14698557460913433 0.13694577040087055 0.04996654452682453
322
size 52
0.04754790538241596 1.0081862607814258
0.22797880502184853 0.1469120041676797 0.04754790538241596 0.05472559925488985
InnerProd [[ 1.0000001  -0.99612886 -0.9462755 ]
 [-0.99612886  1.0000001   0.9676915 ]
 [-0.9462755   0.9676915   0.99999994]]
retraining
1
Final loss: 80.02714538574219
parameters 31250
323
size 52
0.049109384681497885 1.0081862607814258
0.1719372530351393 0.15074506736826151 0.049109384681497885 0.053991044708445006
324
size 52
0.012443750177390097 1.0081862607814258
0.1316617539850995 0.15089344640960917 0.012443750177390097 0.05312742622446746
325
size 52
0.092711596231377 1.0081862607814258
0.13278015999821946 0.15251977768726646 0.092711596231377 0.053047283532920195
326
size 52
0.01205855536280478 1.0081862607814258
0.11396725900704041 0.15071110708639027 0.01205855536280478 0.053772112230678244
327
size 52
0.0512346207602

372
size 52
0.10906813233173107 1.0081862607814258
0.09116845298558474 0.16630665647564455 0.10906813233173107 0.04640706714454395
373
size 52
0.0947271693614966 1.0081862607814258
0.11267664801562205 0.1645809134352021 0.0947271693614966 0.050378181747100736
374
size 52
0.09309718054711393 1.0081862607814258
0.08983100700424984 0.16367629223503172 0.09309718054711393 0.05198362735235406
375
size 52
0.035692210476546284 1.0081862607814258
0.08986751799238846 0.15722076011588798 0.035692210476546284 0.05499424818938337
376
size 52
0.03902052240627429 1.0081862607814258
0.17048148799221963 0.15384843427687883 0.03902052240627429 0.05383531011210931
377
size 52
0.048656413144401146 1.0081862607814258
0.09112743701552972 0.14972259011585265 0.048656413144401146 0.05363311791619276
378
size 52
0.049489759003663605 1.0081862607814258
0.08995639503700659 0.14711171727627514 0.049489759003663605 0.05452940992478835
379
size 52
0.03273544107897451 1.0081862607814258
0.14987837895751 0.144553687

In [ ]:
import matplotlib.pyplot as pp
from src_dir import *

In [ ]:
pp.plot(run_time_ML_listP2[3:],'.k',GmresRunTimeOriginal_2[3:],'.r')

In [ ]:
AVG=np.zeros((n_steps,1))
Err_Array=np.asarray(NonML_Err_List)
count=np.arange(0,n_steps)

for j in range(0,n_steps):
    AVG[j]=moving_average(np.asarray(Err_Array[:j]),j)

    
    
pp.plot(count,np.asarray(NonML_Err_List),'.r',count[10:-1],AVG[10:-1],'y')
pp.xlabel('$i$')
pp.ylabel('2-norm of residual of 10th iteration')
pp.title('$n=100$ GMRES')
pp.savefig('GMRES.png')

In [ ]:
Err_Array_ML=np.asarray(Err_list)
AVGML=np.zeros((n_steps,1))

for j in range(0,n_steps):
    AVGML[j]=moving_average(np.asarray(Err_Array_ML[:j]),j)

pp.plot(count,np.asarray(Err_Array_ML),'.b',count[10:-1],AVGML[10:-1],'g')
pp.xlabel('$i$')
pp.ylabel('2-norm of residual of 10th iteration')
pp.title('$n=100$ MLGMRES')
pp.savefig('MLGMRES.png')

In [ ]:
pp.plot(count,Err_Array_ML,'.b',label='MLGMRES error')
pp.plot(count[10:-1],AVGML[10:-1],'k',label='Average MLGMRES error')
pp.plot(count,Err_Array,'.r',label='GMRES error')
pp.plot(count[10:-1],AVG[10:-1],'g',label='Average GMRES error')

pp.xlabel('$i$')
pp.ylabel('2-norm of residual of 10th iteration')
pp.title('Error as a function of $b_i$ iteration ')
pp.legend(loc='best')
pp.savefig('Compare.png')

In [ ]:
GmresRunTimeOriginal_AVG=np.zeros((n_steps,1))
ML_GMRES_Time_AVG=np.zeros((n_steps,1))


for j in range(0,n_steps):
    GmresRunTimeOriginal_AVG[j]=moving_average(np.asarray(GmresRunTimeOriginal[:j]),j)

for j in range(0,n_steps):
    ML_GMRES_Time_AVG[j]=moving_average(np.asarray(ML_GMRES_Time_list[:j]),j)
    
    
pp.plot(ML_GMRES_Time_list,'.b',label='MLGMRES')
pp.plot(GmresRunTimeOriginal,'.r', label='GMRES')
pp.plot(count[10:-1],GmresRunTimeOriginal_AVG[10:-1],'g', label='GMRES Average')
pp.plot(count[10:-1],ML_GMRES_Time_AVG[10:-1],'k', label='MLGMRES Average')




pp.ylabel('Time (s)')
pp.xlabel('i')
pp.title('$e1$ tolerance GMRES run time, $n=100$')
pp.legend(loc='best')
pp.savefig('Runtime.png')